# レッスン 01  Predict-Next-Token

このチュートリアルでは、言語モデルの基本メカニズムである「次のトークン予測」（Predict-Next-Token）を深く理解していきます。ゼロから文字レベルの言語モデルを構築し、シェイクスピアの文体を学習させます。この実践プロジェクトを通じて、ニューラルネットワークがどのようにテキストパターンを学習し、特定のスタイルのテキストを生成するかを体験できます。

## Section 1 シェイクスピアテキストデータセット
言語モデルの構築を始める前に、まずトレーニングデータを準備する必要があります。ここでは古典的な「Tiny Shakespeare」データセット——シェイクスピアの全作品を含むテキストコレクションを使用します。

### なぜこのデータセットを選ぶのか？

* 独特なテキストスタイル：シェイクスピアの古英語の文体は特徴的で、モデルの学習効果を観察しやすい
* 適度なデータ量：約1MBのテキストで、高速なトレーニングと実験に適している
* 古典的な教育事例：言語モデル入門チュートリアルで広く使用されている

### データセットのダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

このコマンドでGitHubからinput.txtファイルを現在のディレクトリにダウンロードします。

### テキストデータの読み込み

このコードは、テキストファイルを読み込み、データセットの規模を確認する方法を示しています。

In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

with open() ステートメント

* with はPythonのコンテキストマネージャーで、ファイルを自動的に閉じ、リソースリークを防ぎます
* エラーが発生しても、ファイルは正しく閉じられます
* "r": 読み取りモード（read mode）

### テキストの冒頭を確認
テキストの冒頭を確認任意のデータセットを処理する前に、実際の内容をプレビューすることは非常に重要なステップです。

In [ ]:
print(text[:200])

## Section 2 Pytorchをインストールする

PyTorchをインストールする前に、コンピュータのGPU構成を確認する必要があります。ニューラルネットワークのトレーニング時、GPUは計算速度を大幅に向上させます（通常CPUの100倍以上速い）。

### ステップ1：GPUとCUDAバージョンの確認

ターミナルまたはJupyter Notebookで以下のコマンドを実行します：

In [ ]:
!nvidia-smi

### コマンドの説明

nvidia-smi (NVIDIA System Management Interface)

* NVIDIAが公式に提供するGPU監視ツール

* GPUモデル、ドライババージョン、CUDAバージョン、VRAMの使用状況などの重要な情報を表示

### 出力例の解説

最初の行：ドライバとCUDA情報

* NVIDIA-SMI 580.65.06：NVIDIAドライババージョン
* Driver Version: 580.65.06：ドライバプログラムバージョン
* CUDA Version: 13.0：最重要！ サポートされる最高CUDAバージョン

GPU詳細情報（GPU毎に1行）

* GPU 0, 1：2枚のGPUが検出されました
* NVIDIA H100 80GB HBM3：GPUモデル
* 26C：現在の温度26℃
* 69W / 700W：現在の消費電力/最大消費電力
* 4MiB / 81559MiB：VRAM使用状況 - 使用中4MB / 合計約80GB
* 0%：GPU利用率

### ステップ2: PyTorchのインストール

PyTorch公式サイトのインストールページにアクセス https://pytorch.org/

環境に応じて適切な設定を選択します：

![install_pytorch](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/install_pytorch.png?raw=true)

### インストールの確認

インストール完了後、以下のコードを実行して確認：

In [ ]:
import torch

# PyTorchバージョンの確認
print(f"PyTorchバージョン: {torch.__version__}")

# CUDAが利用可能か確認
print(f"CUDAが利用可能: {torch.cuda.is_available()}")

# CUDAが利用可能な場合、GPU情報を表示
if torch.cuda.is_available():
    print(f"利用可能なGPU数: {torch.cuda.device_count()}")
    print(f"現在のGPU: {torch.cuda.get_device_name(0)}")